In [1]:
import genjax
from genjax import dippl
from genjax import gensp
import jax
import jax.numpy as jnp
import adevjax

console = genjax.pretty(show_locals=True)
key = jax.random.PRNGKey(314159)


# Define our model, and lift to a GenSP choice map distribution.
@genjax.gen
def model():
    x = dippl.normal_reparam(0.0, 10.0) @ "x"
    y = dippl.normal_reparam(0.0, 10.0) @ "y"
    z = dippl.normal_reparam(x**2 + y**2, 0.1) @ "z"


lifted_model = gensp.choice_map_distribution(
    model,
)

# Now, we define our variational proposal.
@genjax.gen
def parametrized_proposal(μ1, σ1, μ2, σ2):
    x = dippl.normal_reparam(μ1, jnp.exp(σ1)) @ "x"
    y = dippl.normal_reparam(μ2, jnp.exp(σ2)) @ "y"


# We embed our proposal into _differentiable_ sampling importance resampling.
variational_sir = dippl.importance_enum(
    100,
    gensp.choice_map_distribution(parametrized_proposal, genjax.select("x", "y")),
)

# Now, we define a grad estimator using the loss language
# (this is lightweight syntax over GenSP's ADEV compatible interfaces
# and ADEV primitives, including `add_cost`)
def elbo_grad(key, fixed_data, params):

    # Loss is defined imperatively using two interfaces:
    # * `upper` invokes `random_weighted` and returns the sample `v`,
    #      and accumulates (-w) via ADEV's `add_cost` primitive.
    #
    # * `lower` invokes `estimate_logpdf` and accumulates (w)
    #      via ADEV's `add_cost` primitive.
    @dippl.loss
    def variational_loss(μ1, σ1, μ2, σ2):
        tgt = gensp.target(model, (), fixed_data)
        v = dippl.do_upper(variational_sir)(tgt, (μ1, σ1, μ2, σ2))
        merged = gensp.merge(v, genjax.ValueChoiceMap(fixed_data))
        dippl.do_lower(lifted_model)(merged)

    # We can automatically derive `jvp` and `grad` estimators using ADEV.
    # (p_grad,) = flip_variational_loss.grad_estimate(key, (p,))
    (μ1, σ1, μ2, σ2) = params
    params_grad = variational_loss.grad_estimate(key, (μ1, σ1, μ2, σ2))
    return params_grad


# Trials:
jitted = jax.jit(elbo_grad)
fixed_data = genjax.choice_map({"z": 5.0})
μ1, σ1, μ2, σ2 = (2.0, 0.01, 2.0, 0.01)


def train(key, fixed_data, params, num_loops=100):
    def _inner(carry, xs):
        (μ1, σ1, μ2, σ2) = carry
        key = xs
        key, sub_key = jax.random.split(key)
        (μ1_grad, σ1_grad, μ2_grad, σ2_grad) = elbo_grad(
            key, fixed_data, (μ1, σ1, μ2, σ2)
        )

        # Nothing too fancy when it comes to optimizer -- just fixed learning rates.
        μ1 = μ1 + 1e-6 * μ1_grad
        σ1 = σ1 + 1e-6 * σ1_grad
        μ2 = μ2 + 1e-6 * μ2_grad
        σ2 = σ2 + 1e-6 * σ2_grad
        return (μ1, σ1, μ2, σ2), (μ1, σ1, μ2, σ2)

    keys = jax.random.split(key, num_loops)
    _, (μ1, σ1, μ2, σ2) = jax.lax.scan(_inner, params, keys)
    return (μ1, σ1, μ2, σ2)


key, sub_key = jax.random.split(key)
jitted = jax.jit(train)
params = jitted(key, fixed_data, (μ1, σ1, μ2, σ2))
params


(
    Array([1.7186848, 1.6114966, 1.5304842, 1.4996581, 1.4640118, 1.4495015,
       1.4263766, 1.4083099, 1.4019818, 1.3983003, 1.3956025, 1.4003799,
       1.4070473, 1.4110829, 1.4112598, 1.4093934, 1.4073944, 1.4187672,
       1.4126804, 1.4144053, 1.4214003, 1.4298216, 1.4375436, 1.4436388,
       1.4502875, 1.4568324, 1.467121 , 1.4691129, 1.4722154, 1.4694514,
       1.4781789, 1.4855672, 1.4879848, 1.4917557, 1.4945   , 1.4978713,
       1.4926543, 1.5013129, 1.4995786, 1.5050484, 1.5100108, 1.5134976,
       1.5174713, 1.5186758, 1.5209107, 1.5285653, 1.5355713, 1.5303237,
       1.5360943, 1.5450127, 1.5410334, 1.5419798, 1.5540276, 1.5529407,
       1.549527 , 1.5513718, 1.547248 , 1.5492202, 1.558666 , 1.5567755,
       1.5516461, 1.545857 , 1.5510546, 1.5534697, 1.5500288, 1.5505124,
       1.5563114, 1.5582879, 1.5582442, 1.5532975, 1.5488502, 1.553396 ,
       1.5559819, 1.5579915, 1.55898  , 1.5634419, 1.56482  , 1.5625387,
       1.5627668, 1.5581151, 1.5626724, 1.55

In [2]:
%%timeit
params = jitted(key, fixed_data, (μ1, σ1, μ2, σ2))

9.49 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
